In [1]:
import os
import openai
import json
import pandas as pd
from transformers import GPT2TokenizerFast
import nltk

openai authentication

In [2]:
openai.organization = "org-us16wmNswbfs7htSVY2eiaYh" # zazu
openai.api_key = os.getenv("OPENAI_API_KEY")

read in the protocol in csv format and convert to string

In [4]:
data = pd.read_csv('text/clinical_trial_rank_0131/text.csv')
# data = pd.read_csv('cut_trial0061.csv')
text = data.astype(str)

In [4]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

## Chunks

Cut the document into chunks.
1. Start with question
2. Keep adding until we get 3500 tokens
3. Send the chunk to API

In [6]:
# question = """Extract the validated clinical questionnaires used in this text in a comma separated list with each questionnaire on a new line:
# example text: nation', 'EMA', 'European Medicines Agency', 'DLQI', 'Dermatology Life Quality Index', 'EQ-5D', 'European Quality of Life 5-Dimension Questionnaire', 'FCBP', 'Females of childbearing potential', 'FDA', 'Food and Drug Administration'
# example answer: 
# - EQ-5D, European Quality of Life 5-Dimension Questionnaire
# - DLQI, Dermatology Life Quality Index
# """

In [7]:
# question = """Extract the validated clinical questionnaires used in this text with a dash, long name and short name in brackets like so:
# - Patient Benefit Index, (PBI)
# - European Quality of Life 5-Dimension Questionnaire, (EQ-5D)
# """

In [8]:
question = """Extract the validated clinical questionnaires used in this text in a comma separated list with each questionnaire on a new line. An example format is shown:
example text: nation', 'EMA', 'European Medicines Agency', 'DLQI', 'Dermatology Life Quality Index', 'EQ-5D', 'European Quality of Life 5-Dimension Questionnaire', 'FCBP', 'Females of childbearing potential', 'FDA', 'Food and Drug Administration'
example answer: 
- EQ-5D, European Quality of Life 5-Dimension Questionnaire
- DLQI, Dermatology Life Quality Index
"""

In [18]:
chunks = []
page = 0

while page < len(text):
    page_count = 0
    chunk_tokens = 0
    chunk = ""
    while chunk_tokens <= 3500 and page_count < 5 and page < len(text):
        chunk += text.iloc[page][2]
        chunk_tokens += len(tokenizer(chunk)[0])
        page += 1
        page_count += 1
    chunks.append(chunk)

In [8]:
# chunks4 = []
# page = 0

# while page < len(text):
#     page_count = 0
#     chunk_tokens = 0
#     chunk = ""
#     while chunk_tokens <= 30000 and page_count < 25 and page < len(text):
#         chunk += text.iloc[page][2]
#         chunk_tokens += len(tokenizer(chunk)[0])
#         page += 1
#         page_count += 1
#     chunks4.append(chunk)

In [ ]:
            {"role": "system", "content": """You are a helpful assistant that extracts validated clinical questionnaires and puts them into comma separated list with each questionnaire on a new line. Do not ask questions.
            If you can't find any questionnaires say: 'None found'.
            example text: nation', 'EMA', 'European Medicines Agency', 'DLQI', 'Dermatology Life Quality Index', 'EQ-5D', 'European Quality of Life 5-Dimension Questionnaire', 'FCBP', 'Females of childbearing potential', 'FDA', 'Food and Drug Administration'
            example answer: 
            - EQ-5D, European Quality of Life 5-Dimension Questionnaire
            - DLQI, Dermatology Life Quality Index"""},

### ChatGPT API


In [25]:
answers = []
for chunk in chunks:
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": """You are a helpful assistant that extracts validated clinical questionnaires and puts them into comma separated list with each questionnaire on a new line. Do not ask questions.
            If you find a questionnaire tell me the timeframe it is being used.
            If you can't find any questionnaires say: 'None found'. 
            If you can't find the timing of the questionnaires say: 'Week/month not specified'. 
            """},
            {"role": "user", "content": chunk},
        ]
    )
    answer = completion["choices"][0]["message"]["content"]
    answers.append(answer)

In [24]:
for answer in answers:
    print(answer)

None found.
MSQOL-54 questionnaire
Week/month not specified
None found.
None found.
Week/month not specified. None found. 

(Note: There are several questionnaires listed, however, they are not validated clinical questionnaires, but rather assessments and surveys used to collect data on various aspects such as smoking status, medication use, comorbidities, BMI, perceived social support, mental and physical quality of life, fatigue, anxiety and depression, patient-reported disability, and self-efficacy.)
None found.
None found.
MSQOL-54 - Week/month not specified\
PDDS - Week/month not specified\
HADS - Week/month not specified\
FSS - Week/month not specified\
UWSE-6 - Week/month not specified\
IPAQ-SF - Week/month not specified\
MAQ - Week/month not specified\
DHQ - Week/month not specified\
Vitamin D intake - Week/month not specified\
Perceived Social Support - Week/month not specified\
None found.
MSQOL-54 - Week not specified. None found for specific timing.
MSQOL-54 questionnaire -

['None found.',
 'None found.',
 '- FACT-G, Functional Assessment of Cancer Therapy-General',
 'None found.',
 'None found.',
 'None found.',
 'None found.',
 'None found.',
 'None found.',
 'None found.',
 'None found.',
 'None found.',
 'None found.',
 'None found.',
 '- FACT-G QUESTIONNAIRE, VERSION 4, ENGLISH AND SWAHILI\nNone found\n- FACT-G (Version 4), SWAHILI',
 'None found.']

## gpt4

In [26]:
text = '''	 	Pre- Study	Cycle 1	Cycle 2	Cycle 3	Cycle 4	Cycle 5	Cycle 6	Treatment Discontinuation	Off Study
1	Informed consent	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
2	Demographics	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
3	Medical history	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
4	Physical exam	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X
5	Vital signs	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X
6	Height	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
7	Weight	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X	NOT_SELECTED	SELECTED X	NOT_SELECTED	SELECTED X	SELECTED X
8	Biopsy diagnostic of KS (fixed in formalin)	SELECTED Xe	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
9	FACT-G Questionnaire	NOT_SELECTED	SELECTED Xf	NOT_SELECTED	NOT_SELECTED	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X
10	Toxicity evaluations	NOT_SELECTED	SELECTED	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X
11	Performance status	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X
12	CBC w/diff, plts	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED	SELECTED X	SELECTED X
13	Serum chemistry	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X
14	Oxygen saturation	SELECTED Xb	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X
15	Radiologic evaluation	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X
16	Tumor measurements	SELECTED X	NOT_SELECTED	SELECTED X	NOT_SELECTED	SELECTED X	NOT_SELECTED	SELECTED X	SELECTED X	SELECTED X
17	Pregnancy test	SELECTED Xd	NOT_SELECTED	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	NOT_SELECTED	NOT_SELECTED
'''

In [ ]:
You are a helpful assistant that extracts validated clinical questionnaires and puts them into comma separated list with each questionnaire on a new line. Do not ask questions.
            If you find a questionnaire tell me the timeframe it is being used.
            If you can't find any questionnaires say: 'None found'. 

In [28]:
text = """	Study Period	 	 	 	 	 	Maintenance	 	 	 	 	Unsched [1]	ED [2]	Exit [2]
1	Study Visit (Week)	1, 5, 9	13	17,21	25	OLFC [3]	RWPC [4]	29	33, 37	41	45,49	na	na	53
2	Interval	 	 	 	 	Every	4 weeks	 	 	 	 	Varies	Varies	na
3	Window (Days)	 	 	 	 	 	7	 	 	 	 	na	3	7
4	HRQOL Questionnaires and Bespoke Questions [5]	 	 	 	 	 	 	 	 	 	 	 	 	 
5	EQ-5D [6]	NOT_SELECTED	SELECTED X	NOT_SELECTED	NOT_SELECTED	X [6]	X [6]	NOT_SELECTED	NOT_SELECTED	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X [7]	SELECTED X
6	FAQL-PB [8]	NOT_SELECTED	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	 	NOT_SELECTED	NOT_SELECTED	SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X
7	HADS [8]	NOT_SELECTED	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	 	NOT_SELECTED	NOT_SELECTED	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X
8	FAQLQ, FAIM [9]	NOT_SELECTED	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X [9]	X [9	NOT_SELECTED	NOT_SELECTED	SELECTED	NOT_SELECTED	NOT_SELECTED	X [7]	SELECTED X
9	Bespoke global assessment of HRQOL	NOT_SELECTED	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	 	NOT_SELECTED	NOT_SELECTED	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X
10	Bespoke assessment of control and confidence	NOT_SELECTED	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X
11	Bespoke assessment of food challenge outcomes	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED X [10]	X [10]	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
12	TSQM-9	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X
13	Bespoke exit questionnaire	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X
14	Exit interview [11]	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X
15	General Activities	 	 	 	 	 	 	 	 	 	 	 	NOT_SELECTED	NOT_SELECTED
16	Weight, height	SELECTED X	SELECTED X	SELECTED X	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X
17	Vital signs (blood pressure, heart rate, temperature)	SELECTED X	SELECTED	SELECTED	SELECTED	SELECTED X	SELECTED X	SELECTED X	SELECTED	SELECTED	SELECTED	SELECTED X	SELECTED X	SELECTED X
18	ACT/C-ACT, SCORAD, TNSS (if completed at screening)	NOT_SELECTED	SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED	NOT_SELECTED	X (optional)	SELECTED X	SELECTED X
19	PEFR [12]	SELECTED	SELECTED	SELECTED	SELECTED	SELECTED X	SELECTED X	SELECTED	SELECTED	SELECTED	SELECTED	X (optional)	SELECTED X	SELECTED X
20	Asthma evaluation [13]	NOT_SELECTED	SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED	NOT_SELECTED	X (optional)	SELECTED X	SELECTED X
21	Symptom-directed physical examination [14]	SELECTED	SELECTED	SELECTED	SELECTED	SELECTED X	SELECTED X	SELECTED	SELECTED	SELECTED	SELECTED	SELECTED X	NOT_SELECTED	NOT_SELECTED
22	Complete physical examination [14]	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	X (optional)	SELECTED X	SELECTED X
23	Diet/food allergen exposure review	SELECTED	SELECTED	SELECTED	SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED	SELECTED	SELECTED	SELECTED	SELECTED X	SELECTED X	SELECTED X
24	Food allergy instruction [15]	SELECTED	SELECTED	SELECTED	SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED	SELECTED	SELECTED	SELECTED X	X (optional)	SELECTED X	SELECTED X
25	Contraception review	SELECTED X	SELECTED	SELECTED	SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED	SELECTED	SELECTED	SELECTED	X (optional)	NOT_SELECTED	NOT_SELECTED
26	Adverse events review [16]	SELECTED X	SELECTED	SELECTED	SELECTED	SELECTED X	SELECTED X	SELECTED X	SELECTED	SELECTED	SELECTED X	SELECTED X	SELECTED X	SELECTED X
27	Concomitant medications review & instruction [17]	SELECTED X	SELECTED	SELECTED X	SELECTED	SELECTED X	SELECTED X	SELECTED	SELECTED	SELECTED	SELECTED X	SELECTED X	SELECTED X	SELECTED X
28	Study product administration [18]	SELECTED	SELECTED	SELECTED	SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED	SELECTED	SELECTED	SELECTED	X (optional)	NOT_SELECTED	NOT_SELECTED
"""

In [30]:
text = """	Assessment	Source Data Type	Baseline 1 w/in 2 mos of phone screen & 14 wks of intervention start	Baseline 2 w/in 6 wks of starting intervention	Follow-up 1 0-7 days after intervention completion	Follow-up 2 1-21 days after intervention completion
1	Alternative Uses Task	Paper	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED
2	Apathy Evaluation Scale (AES)	Paper	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED
3	Blood draw - biomarkers	Lab Report	SELECTED X	NOT_SELECTED	SELECTED X	NOT_SELECTED
4	Consent	Paper	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
5	Contact Form	Paper	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
6	Demographics	Paper	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
7	Expanded Short Physical Performance Battery	Paper	SELECTED X	NOT_SELECTED	SELECTED X	NOT_SELECTED
8	Falls Efficacy Scale	Paper	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED
9	Fullerton Advanced Balance Scale	Paper	SELECTED X	NOT_SELECTED	SELECTED X	NOT_SELECTED
10	Gait Speed/Variability	Electronic	SELECTED X	NOT_SELECTED	SELECTED X	NOT_SELECTED
11	Geriatric Anxiety Scale	Paper	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED
12	Geriatric Depression Scale	Paper	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED
13	Medical History/RX	Paper	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
14	MoCA	Paper	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X
15	Multisensory Reaction Time	Electronic	NOT_SELECTED	SELECTED X	NOT_SELECTED	SELECTED X
16	NEO-FFI	Paper	NOT_SELECTED	SELECTED X	NOT_SELECTED	NOT_SELECTED
17	NPI-Q	Paper	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED
18	Philadelphia Mindfulness Scale	Paper	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X
19	Postural Sway	Electronic	SELECTED X	NOT_SELECTED	SELECTED X	NOT_SELECTED
20	RBANS	Paper	NOT_SELECTED	SELECTED X	NOT_SELECTED	SELECTED X
21	SF-36	Paper	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED
22	Study Expectations	Paper	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
23	Timed Up and Go	Paper	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X
24	Zarit Caregiver Burden Scale	Paper	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED
25	Alternative Uses Task	Paper	NOT_SELECTED	SELECTED X	NOT_SELECTED	SELECTED X
26	Apathy Evaluation Scale	Paper	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED
27	Blood draw - biomarkers	Lab Report	SELECTED X	NOT_SELECTED	SELECTED X	NOT_SELECTED
28	Consent	Paper	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
29	Contact Form	Paper	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
30	Demographics	Paper	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
31	Expanded Short Physical Performance Battery	Paper	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X
32	Falls Efficacy Scale	Paper	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED
33	Fullerton Advanced Balance Scale	Paper	SELECTED X	NOT_SELECTED	SELECTED X	NOT_SELECTED
34	Gait Speed/Variability	Electronic	SELECTED X	NOT_SELECTED	SELECTED	SELECTED X
35	Geriatric Anxiety Scale	Paper	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED
36	Geriatric Depression Scale	Paper	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED
37	Medical History/RX	Paper	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
38	MoCA	Paper	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X
39	MRI	Electronic	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED
40	Multisensory Reaction Time	Electronic	NOT_SELECTED	SELECTED X	NOT_SELECTED	SELECTED X
41	Philadelphia Mindfulness Scale	Paper	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X
42	Postural Sway	Electronic	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X
43	QOL-AD	Paper	SELECTED X	NOT_SELECTED	SELECTED X	NOT_SELECTED
44	RBANS	Paper	NOT_SELECTED	SELECTED X	NOT_SELECTED	SELECTED X
45	Study Expectations	Paper	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
46	Timed Up and Go	Paper	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X
"""

In [6]:
69
text = """									
									
				Cycle 1			Prior to Day 1 of subsequent cycles3	Every 2 months	End of Treatement
tests and procedures	<28 days prior to registration	<14 days prior to registration	<7 days prior to registration	D1	D3	D10 			
window:							±3 days	±7 days	±14 days
PET-CT or CT (Chest. Abdomen and pelvis), or MRI abdomen MRI neck, if needed 	X							X	X
ECG							X		
2D-ECHO	X								
Blood or Tissue- Analysis performed by Research Lab									
Optional tissue sample for correlatives (see Section 17.0)13 X Optional research blood will be collected under IRB 13-00583814 			X						
Quality of Life assessment		X							
Patient Questionnaire Booklet: EQ-5D-5L FACT-G			X				X		
"""

In [7]:
61
text = """	Protocol Activity	Early Treatment Discontinuation Visit of		Day 1 through Week 28	
1	(Note: All visit timings are relative to Day 1)	(within 2 weeks discontinuing study treatment)	Week 4, 16, 28 2 weeks	Week 24	Unscheduled
2	IRT system call	X	SELECTED X	SELECTED X	SELECTED X
3	SBP/DBP, HR	X (Triplicate)	NOT_SELECTED	NOT_SELECTED	SELECTED X
4	ECG	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
5	Rescue medication 1,2	SELECTED X	SELECTED X	SELECTED X	NOT_SELECTED
6	Urine (serum if transitioned to dialysis) pregnancy test (WOCBP only)	SELECTED X7	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
7	HemoCue Hgb	SELECTED X	SELECTED X	SELECTED X	SELECTED X
8	Hematology³	SELECTED X	SELECTED X	Hgb Only	NOT_SELECTED
9	Clinical chemistry³	SELECTED X	SELECTED X	NOT_SELECTED	NOT_SELECTED
10	Iron Panel	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
11	Hospitalization¹, transition to dialysis¹	SELECTED X	SELECTED X	SELECTED X	SELECTED X
12	hsCRP	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
13	Adverse event assessment	SELECTED X	SELECTED X	SELECTED X	SELECTED X
14	Review concomitant medications	SELECTED X	SELECTED X	SELECTED X	SELECTED X
15	Actigraphy 4	SELECTED X9	SELECTED X	NOT_SELECTED	NOT_SELECTED
16	CKD-AQ8	SELECTED X	SELECTED X	NOT_SELECTED	NOT_SELECTED
17	PGI-S, PGI-C 8	SELECTED X	SELECTED X	NOT_SELECTED	NOT_SELECTED
18	SF-36 8	SELECTED X	SELECTED X	NOT_SELECTED	NOT_SELECTED
19	EQ-5D-5L& EQ-VAS 6,8	SELECTED X	SELECTED X	NOT_SELECTED	NOT_SELECTED
20	WPAI-ANS-CPV 5,8	SELECTED X	SELECTED X	NOT_SELECTED	NOT_SELECTED
21	ADPKD participant only: Ultrasound	SELECTED X10	NOT_SELECTED		
22	Healthcare resource utilization (participant- reported)	X	NOT_SELECTED		"""

In [9]:
27
text = """	Study Period	,Initial Dose	,Escalation		,Up-Dosing			,Early
1	Study Visit	Day 1 [1]	Day 2 [2]	At Study Site	At 80 mg	At 300 mg	Unsched [3]	Disc [4]
2	Interval	na	na	Every 2 weeks	Varies	Varies	Varies	Varies
3	Window (Days)	na	na	3	+3	+3	na	+3
4	HRQOL Questionnaires and Bespoke Questions [5]							
5	EQ-5D [6]	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED	x[7]
6	FAQL-PB [8]	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED X
7	HADS [8]	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED X
8	FAQLQ, FAIM [9]	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED	SELECTED x[7]
9	Bespoke global assessment of HRQOL	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED X
10	Bespoke assessment of control and confidence	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED X
11	Randomization	SELECTED X	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED
12	General Activities							NOT_SELECTED
13	Weight, height	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X
14	Vital signs (blood pressure, heart rate, temperature)	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X
15	ACT/C-ACT, SCORAD, TNSS (if completed at screening)	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X	X (optional)	SELECTED X
16	PEFR [10]	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X (optional)	SELECTED X
17	Asthma evaluation [11]	NOT_SELECTED	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X	SELECTED X (optional)	SELECTED X
18	Symptom-directed physical examination [12]	NOT_SELECTED	SELECTED X	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X	NOT_SELECTED
19	Complete physical examination [12]	SELECTED X	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X	X (optional)	SELECTED X
20	Diet/food allergen exposure review	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X
21	Food allergy instruction [13]	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	X (optional)	SELECTED X
22	Contraception review	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X (optional)	NOT_SELECTED
23	Adverse events review [14]	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X
24	Concomitant medications review & instruction [15]	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X
25	Study product administration [16]	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X	X (optional)	NOT_SELECTED
26	Study product dispensing [17]	NOT_SELECTED	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X (optional)	NOT_SELECTED
27	Study product accountability	NOT_SELECTED	NOT_SELECTED	SELECTED X	SELECTED X	SELECTED X	SELECTED X (optional)	SELECTED X
28	Telephone call [18]	NOT_SELECTED	SELECTED X	SELECTED X	SELECTED X	SELECTED X	SELECTED X (optional)	NOT_SELECTED"""

In [8]:
131
text = """							Timeframe		
1	Assessment/Measure	No. Questions	Screening	Baseline	Complete Online course	One month after Online course	6 months after Online course	12 months after Online course	2.5 years after Online course
2	Informed consent	1		SELECTED X					
3	Age	1	SELECTED X	SELECTED X					
4	MS type diagnosed	2	SELECTED X	SELECTED X			SELECTED X	SELECTED X	SELECTED X
5	MS duration	2	SELECTED X	SELECTED X			SELECTED X	SELECTED X	SELECTED X
6	Do you follow a MS-specific lifestyle program?	2	SELECTED X	SELECTED X			SELECTED X	SELECTED X	SELECTED  X
7	Do you follow a MS-specific diet	2	SELECTED X	SELECTED X			SELECTED X	SELECTED X	SELECTED X
8	Sex and gender	2		SELECTED X					
9	Residential address and country	1		SELECTED X					SELECTED X
10	Country of birth	1		SELECTED X					
11	Height/weight	2		SELECTED X			SELECTED X	SELECTED X	SELECTED X
12	Comorbidities	2		SELECTED X			SELECTED X	SELECTED X	SELECTED X
13	Marital status	1		SELECTED X			SELECTED X	SELECTED X	SELECTED X
14	Education	1		SELECTED X					
15	Employment status	1		SELECTED X			SELECTED X	SELECTED X	SELECTED X
16	Alcohol and smoking	4		SELECTED X			SELECTED X	SELECTED X	SELECTED X
17	Medications	4		SELECTED X			SELECTED X	SELECTED X	SELECTED X
18	Perceived Social Support	12		SELECTED X			SELECTED X	SELECTED X	SELECTED X
19	Lifestyle factors								
20	Physical activity: IPAQ-SF	7		SELECTED X			SELECTED X	SELECTED X	SELECTED X
21	Meditation: MAQ	3		SELECTED X			SELECTED X	SELECTED X	SELECTED X
22	Sun exposure	4		SELECTED X			SELECTED X	SELECTED X	SELECTED X
23	Diet quality: DHQ	21		SELECTED X			SELECTED X	SELECTED X	SELECTED X
24	Omega-3 intake: dose, frequency	3		SELECTED X			SELECTED X	SELECTED X	SELECTED X
25	Vitamin D intake: dose, frequency	3	SELECTED  X			SELECTED  X	SELECTED X	SELECTED X	SELECTED X
26	Health outcomes								
27	HRQOL: MSQOL-54	54	SELECTED X			SELECTED X	SELECTED X	SELECTED X	
28	Disability: PDDS	1	SELECTED X			SELECTED X	SELECTED X	SELECTED X	
29	Anxiety and depression: HADS	14	SELECTED X			SELECTED X	SELECTED X	SELECTED X	
30	Fatigue: FSS	9	SELECTED X			SELECTED  X	SELECTED X	SELECTED  X	
31	Self-efficacy: UWSE-6	6	SELECTED X			SELECTED X	SELECTED X	SELECTED X	
32	Qualitative interviewing	-7		SELECTED X			SELECTED X		
33	If clinically significant improvement								
34	in HRQOL subset scores in								
35	Intervention group from baseline,								
36	we will consider an amendment to the RCT to provide participants in the standard-care access to the intervention course						SELECTED X		
37	If no clinically significant improvement in HRQOL subset scores in Intervention group from baseline, the RCT will not be amended						SELECTED X		
"""

In [13]:
131 # edited
text = """Assessment/Measure	No. Questions	Screening	Baseline	Complete Online course	One month after Online course	6 months after Online course	12 months after Online course	2.5 years after Online course
Informed consent	1	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X
Age	1	SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X
MS type diagnosed	2	SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X
MS duration	2	SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X
Do you follow a MS-specific lifestyle program?	2	SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED  X
Do you follow a MS-specific diet	2	SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X
Sex and gender	2	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X
Residential address and country	1	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X
Country of birth	1	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X
Height/weight	2	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X
Comorbidities	2	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X
Marital status	1	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X
Education	1	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X
Employment status	1	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X
Alcohol and smoking	4	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X
Medications	4	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X
Perceived Social Support	12	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X
Lifestyle factors		NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X
Physical activity: IPAQ-SF	7	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X
Meditation: MAQ	3	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X
Sun exposure	4	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X
Diet quality: DHQ	21	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X
Omega-3 intake: dose, frequency	3	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X
Vitamin D intake: dose, frequency	3	SELECTED  X	NOT SELECTED X	NOT SELECTED X	SELECTED  X	SELECTED X	SELECTED X	SELECTED X
Health outcomes		NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X
HRQOL: MSQOL-54	54	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X	NOT SELECTED X
Disability: PDDS	1	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X	NOT SELECTED X
Anxiety and depression: HADS	14	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X	NOT SELECTED X
Fatigue: FSS	9	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED  X	SELECTED X	SELECTED  X	NOT SELECTED X
Self-efficacy: UWSE-6	6	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	SELECTED X	NOT SELECTED X
Qualitative interviewing	-7	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X

"""

In [5]:
# test
text = """Measure	week 1	week 2	week 5 	week 10 	week 20 	week 35	Post study completion
HRQOL: MSQOL-54	SELECTED X	NOT SELECTED X	SELECTED X	NOT SELECTED X	SELECTED X	NOT SELECTED X	SELECTED X
Disability: PDDS	SELECTED X	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X
Fatigue: FSS	SELECTED X	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X
Anxiety and depression: HADS	SELECTED X	NOT SELECTED X	SELECTED X	NOT SELECTED X	SELECTED X	NOT SELECTED X	SELECTED X
Self-efficacy: UWSE-6	SELECTED X	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X
Age	SELECTED X	NOT SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X
MS type diagnosed	SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X
MS duration	SELECTED X	SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	NOT SELECTED X	SELECTED X
Do you follow a MS-specific lifestyle program?	SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X
Do you follow a MS-specific diet	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X
Sex and gender	SELECTED X	NOT SELECTED X	SELECTED X	SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X
Residential address and country	SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	NOT SELECTED X	SELECTED X

"""

In [ ]:
give me a study schedule in weeks and days based on the following text. 
        Only include validated clinical questionnaire.
        

In [6]:
completion = openai.ChatCompletion.create(
model="gpt-4",
messages=[
        {"role": "system", "content": """give me a study schedule in weeks and days based on the validated clinical questionnaires in the following text.
        """},
        {"role": "user", "content": text},
    ]
)
answer = completion["choices"][0]["message"]["content"]
print(answer)

Study Schedule

Week 1:
1. HRQOL: MSQOL-54
2. Disability: PDDS
3. Fatigue: FSS
4. Anxiety and Depression: HADS
5. Self-efficacy: UWSE-6
6. Age
7. MS type diagnosed
8. MS duration
9. Do you follow an MS-specific lifestyle program?
10. Do you follow an MS-specific diet?
11. Sex and gender
12. Residential address and country

Week 2:
1. MS type diagnosed

Week 5:
1. HRQOL: MSQOL-54
2. Disability: PDDS
3. Fatigue: FSS
4. Anxiety and Depression: HADS
5. Self-efficacy: UWSE-6
6. Age
7. MS type diagnosed
8. Do you follow an MS-specific lifestyle program?
9. Sex and gender

Week 10:
1. HRQOL: MSQOL-54
2. MS duration
3. Do you follow an MS-specific diet?
4. Sex and gender

Week 20:
1. HRQOL: MSQOL-54
2. MS duration
3. Do you follow a MS-specific diet?

Week 35:
1. Anxiety and Depression: HADS
2. Residential address and country

Post-Study Completion:
1. HRQOL: MSQOL-54
2. Fatigue: FSS
3. Anxiety and Depression: HADS
4. Age
5. MS type diagnosed
6. MS duration
7. Do you follow an MS-specific life

131 (edited)

Week 1:
- Day 3: Physical activity: IPAQ-SF, Meditation: MAQ,Diet quality: DHQ, 

Week 2:
- Day 1: Health outcomes, HRQOL: MSQOL-54, Disability: PDDS, Anxiety and depression: HADS, Fatigue: FSS, and Self-efficacy: UWSE-6.

Week 4:
- Day 1: One month after Online course, repeat HRQOL: MSQOL-54, Disability: PDDS, Anxiety and depression: HADS, Fatigue: FSS, and Self-efficacy: UWSE-6.

Week 24 (6 months after Online course):
- Day 1: Physical activity: IPAQ-SF, Meditation: MAQ, Sun exposure, Diet quality: DHQ
- Day 3: Disability: PDDS, Anxiety and depression: HADS, Fatigue: FSS, and Self-efficacy: UWSE-6.

Week 48 (12 months after Online course):
- Day 1: Physical activity: IPAQ-SF, Meditation: MAQ, Diet quality: DHQ
- Day 3: Disability: PDDS, Anxiety and depression: HADS, Fatigue: FSS, and Self-efficacy: UWSE-6.

Week 130 (2.5 years after Online course):
- Day 1: Physical activity: IPAQ-SF


131
Week 1: Day 1
- Assessment tasks:
    1. Informed consent
    2. Age
    3. MS type diagnosed
    4. MS duration
    5. Do you follow an MS-specific lifestyle program?
    6. Do you follow an MS-specific diet?
    7. Sex and gender
    8. Residential address and country
    9. Country of birth
    10. Height/weight
    11. Comorbidities
    
Week 1: Day 2
- Assessment tasks:
    12. Marital status
    13. Education
    14. Employment status
    15. Alcohol and smoking
    16. Medications
    17. Perceived Social Support
    18. Physical activity: IPAQ-SF
    19. Meditation: MAQ

Week 1: Day 3
- Assessment tasks:
    20. Sun exposure
    21. Diet quality: DHQ
    22. Omega-3 intake: dose, frequency
    23. Vitamin D intake: dose, frequency
    
Week 1: Day 4
- Clinical Validated Questionnaire Day
    - Health outcomes:
        24. HRQOL: MSQOL-54
        25. Disability: PDDS
        26. Anxiety and depression: HADS
        27. Fatigue: FSS
        28. Self-efficacy: UWSE-6
        
Week 1: Day 5
- Qualitative interviewing

Baseline Period (Variable in length)

Complete Online Course:
- Weekly update and check-in with the participant
- Monitor progress and address any concerns or questions

One Month after Online Course:
- Repeat assessment tasks 4-7, 11-12, 15-17, and 19-22

Six Months after Online Course:
- Repeat assessment tasks 4-7, 11-12, 15-17, and 19-22

Twelve Months after Online Course:
- Repeat assessment tasks 4-7, 11-12, 15-17, and 19-22
- Update residential address and country (Task 9)

Clinical Validated Questionnaire Day (2.5 Years after Online Course):
- Repeat clinical validated questionnaire, tasks 24-28


In [8]:
answer

'Week 1:\n- Day 1: Clinical validated questionnaire: EQ-5D-5L FACT-G\n- Day 1-7: ECG test\n- Day 3: Optional research blood collection under IRB 13-005838\n- Day 4-10 (±3 days): Blood or Tissue- Analysis performed by Research Lab\n- Day 7: PET-CT or CT (Chest, Abdomen, and pelvis) or MRI abdomen MRI neck (if needed)\n\nWeek 2-4:\n- (Within 14 days prior to registration): Optional tissue sample for correlatives (see Section 17.0)13\n\nWeek 5-8:\n- (Within 28 days prior to registration): 2D-ECHO test\n- Quality of Life assessment\n\nWeek 9-12:\n- Day 1 of subsequent cycles: PET-CT or CT (Chest, Abdomen, and pelvis) or MRI abdomen MRI neck (if needed)\n- Clinical validated questionnaire: EQ-5D-5L FACT-G\n\nEvery 2 months:\n- PET-CT or CT (Chest, Abdomen, and pelvis) or MRI abdomen MRI neck (if needed)\n\nEnd of Treatment:\n- PET-CT or CT (Chest, Abdomen, and pelvis) or MRI abdomen MRI neck (if needed)\n\nPlease note that the schedule may be adjusted with ±7 days or ±14 days window dependi

In [12]:
answers = []
for chunk in chunks:
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": """You are a helpful assistant that extracts validated clinical questionnaires and puts them into comma separated list with each questionnaire on a new line. Do not ask questions.
            If you can't find any questionnaires say: 'None found'.
            example text: nation', 'EMA', 'European Medicines Agency', 'DLQI', 'Dermatology Life Quality Index', 'EQ-5D', 'European Quality of Life 5-Dimension Questionnaire', 'FCBP', 'Females of childbearing potential', 'FDA', 'Food and Drug Administration'
            example answer: 
            - EQ-5D, European Quality of Life 5-Dimension Questionnaire
            - DLQI, Dermatology Life Quality Index"""},
            {"role": "user", "content": chunk},
        ]
    )
    answer = completion["choices"][0]["message"]["content"]
    answers.append(answer)

In [16]:
# answers = ['\n\n- Patient Global Impression of Severity (PGI-S)\n- Patient Global Impression of Change (PGI-C)\n- Symptoms of aCKD questionnaire (SCKDQ)\n- Short Form 36 (SF-36)\n- EuroQol 5 Dimension 5 Level Health Utility Index (EQ-5D-5L)\n- EuroQol Visual Analogue Scale (EQ-VAS)\n- Work Productivity and Activity Impairment Questionnaire (WPAI-ANS-CPV)\n- Healthcare resource utilization (participants-reported)\n- Actigraphy (wearable)',
#  '\n\n- Patient Benefit Index, (PBI)\n- European Quality of Life 5-Dimension Questionnaire, (EQ-5D)\n- Chronic Kidney Disease - Anemia Questionnaire (CKD-AQ)\n- Patient Global Impression of Severity (PGI-S)\n- Patient Global Impression of Change (PGI-C)\n- SF-36\n- EQ-5D-5L\n- EQ-VAS\n- Work Productivity and Activity Impairment - Anemia Specific (WPAI-ANS-CPV)',
#  '\n\n- Patient Benefit Index, (PBI)\n- European Quality of Life 5-Dimension Questionnaire, (EQ-5D)',
#  'No validated clinical questionnaires were used in this text.',
#  'No validated clinical questionnaires were used in this text.',
#  '\n\n- Patient Benefit Index, (PBI)\n- European Quality of Life 5-Dimension Questionnaire, (EQ-5D)',
#  '\n\n- Chronic Kidney Disease - Anemia Questionnaire (CKD-AQ)\n- Patient Global Impression of Severity (PGI-S) and Patient Global Impression of Change (PGI-C)\n- Health Related Quality of Life (SF-36)\n- Health Status (EQ-5D-5L & EQ-VAS)\n- Work Productivity and Activity Impairment (WPAI-ANS-CPV)',
#  '\n\n- Patient Benefit Index, (PBI)\n- European Quality of Life 5-Dimension Questionnaire, (EQ-5D)\n- Chronic Kidney Disease - Anemia Questionnaire, (CKD-AQ)\n- EuroQol 5 Dimension 5 Level Health Utility Index, (EQ-5D-5L)\n- EuroQol Visual Analogue Scale, (EQ-VAS)',
#  '\n\n- Work Productivity and Activity Impairment Questionnaire: Anemic Symptoms Clinical Practice Version, (WPAI-ANS-CPV)\n- Short Form -36, (SF-36)\n- Patient Global Impression of Change, (PGI-C)\n- Patient Global Impression of Severity, (PGI-S)',
#  '\n\n- Patient Benefit Index (PBI)\n- European Quality of Life 5-Dimension Questionnaire (EQ-5D)',
#  '\n\n- Patient Benefit Index (PBI)\n- European Quality of Life 5-Dimension Questionnaire (EQ-5D)']

In [13]:
answers

['None found.',
 'DLQI, Dermatology Life Quality Index\nEQ-5D, European Quality of Life 5-Dimension Questionnaire\nItch Numeric Rating Scale (NRS)\nSkin Discomfort/Pain Visual Analog Scale (VAS)\nBody Surface Area (BSA)\nPsoriasis Area and Severity Index (PASI)\nPatient Benefit Index (PBI)\nThe Work Productivity and Activity Impairment Questionnaire: Psoriasis (WPAI: PSO)',
 '- DLQI, Dermatology Life Quality Index\n- EQ-5D, The European Quality of Life 5-Dimension Questionnaire\n- FCBP, Serum and Urine Pregnancy Tests for Females of Childbearing Potential',
 'None found.',
 'None found',
 'DLQI, Dermatology Life Quality Index\nEQ-5D, European Quality of Life 5-Dimension',
 'DLQI, Dermatology Life Quality Index',
 'None found.',
 '- DLQI, Dermatology Life Quality Index\n- EQ-5D, European Quality of Life 5-Dimension\n- NAPSI, Nail Psoriasis Severity Index\n- PASI, Psoriasis Area Severity Index\n- PNQ, Patient Needs Questionnaire\n- PBQ, Patient Benefit Questionnaire\n- PPPGA, Palmoplanta

### Answers


In [18]:
for answer in answers:
    print(answer)

None found.
None found.
None found.
None found.
None found.
None found.
None found.
None found.
None found.
None found.
- Expanded Short Physical Performance Battery
- Timed Up and Go
- Philadelphia Mindfulness Scale
- Montreal Cognitive Assessment (MoCA)
- Abbreviated cognitive battery (RBANS)
- Falls Efficacy Scale International
- Geriatric Depression Scale
- Geriatric Anxiety Scale
- Apathy Evaluation Scale
- Alternative Uses Task
- Multisensory Reaction Time
- QOL-AD
- Falls Efficacy Scale
- SF-36
- Geriatric Depression Scale
- Geriatric Anxiety Scale
- Apathy Evaluation Scale
- Fullerton Advanced Balance Scale
- Expanded Short Physical Performance Battery
- Zarit Caregiver Burden Scale
- Neuropsychiatric Inventory Questionnaire
- Alternative Uses Task
- Postural Sway
- Gait Speed and Variability
- Timed Up and Go
- Philadelphia Mindfulness Scale
- Abbreviated Cognitive Battery
- Multisensory Reaction Time
- Fullerton Advanced Balance Scale
- Falls Efficacy Scale
- Expanded Short P

In [19]:
all_answers = []
for answer in answers:
    all_answers.append(answer.split("\n"))

In [20]:
all_answers

[['None found.'],
 ['None found.'],
 ['None found.'],
 ['None found.'],
 ['None found.'],
 ['None found.'],
 ['None found.'],
 ['None found.'],
 ['None found.'],
 ['None found.'],
 ['- Expanded Short Physical Performance Battery',
  '- Timed Up and Go',
  '- Philadelphia Mindfulness Scale',
  '- Montreal Cognitive Assessment (MoCA)',
  '- Abbreviated cognitive battery (RBANS)',
  '- Falls Efficacy Scale International',
  '- Geriatric Depression Scale',
  '- Geriatric Anxiety Scale',
  '- Apathy Evaluation Scale',
  '- Alternative Uses Task',
  '- Multisensory Reaction Time',
  '- QOL-AD'],
 ['- Falls Efficacy Scale',
  '- SF-36',
  '- Geriatric Depression Scale',
  '- Geriatric Anxiety Scale',
  '- Apathy Evaluation Scale',
  '- Fullerton Advanced Balance Scale',
  '- Expanded Short Physical Performance Battery',
  '- Zarit Caregiver Burden Scale',
  '- Neuropsychiatric Inventory Questionnaire',
  '- Alternative Uses Task',
  '- Postural Sway',
  '- Gait Speed and Variability',
  '- Ti

In [21]:
my_answers = []
for lists in all_answers:
    # print(lists)
    for point in lists:
        print(point)
        if point not in my_answers:
            my_answers.append(point)

None found.
None found.
None found.
None found.
None found.
None found.
None found.
None found.
None found.
None found.
- Expanded Short Physical Performance Battery
- Timed Up and Go
- Philadelphia Mindfulness Scale
- Montreal Cognitive Assessment (MoCA)
- Abbreviated cognitive battery (RBANS)
- Falls Efficacy Scale International
- Geriatric Depression Scale
- Geriatric Anxiety Scale
- Apathy Evaluation Scale
- Alternative Uses Task
- Multisensory Reaction Time
- QOL-AD
- Falls Efficacy Scale
- SF-36
- Geriatric Depression Scale
- Geriatric Anxiety Scale
- Apathy Evaluation Scale
- Fullerton Advanced Balance Scale
- Expanded Short Physical Performance Battery
- Zarit Caregiver Burden Scale
- Neuropsychiatric Inventory Questionnaire
- Alternative Uses Task
- Postural Sway
- Gait Speed and Variability
- Timed Up and Go
- Philadelphia Mindfulness Scale
- Abbreviated Cognitive Battery
- Multisensory Reaction Time
- Fullerton Advanced Balance Scale
- Falls Efficacy Scale
- Expanded Short P

In [22]:
my_answers

['None found.',
 '- Expanded Short Physical Performance Battery',
 '- Timed Up and Go',
 '- Philadelphia Mindfulness Scale',
 '- Montreal Cognitive Assessment (MoCA)',
 '- Abbreviated cognitive battery (RBANS)',
 '- Falls Efficacy Scale International',
 '- Geriatric Depression Scale',
 '- Geriatric Anxiety Scale',
 '- Apathy Evaluation Scale',
 '- Alternative Uses Task',
 '- Multisensory Reaction Time',
 '- QOL-AD',
 '- Falls Efficacy Scale',
 '- SF-36',
 '- Fullerton Advanced Balance Scale',
 '- Zarit Caregiver Burden Scale',
 '- Neuropsychiatric Inventory Questionnaire',
 '- Postural Sway',
 '- Gait Speed and Variability',
 '- Abbreviated Cognitive Battery',
 '- MRI scan of brain',
 '- Laboratory Procedures',
 '- IMOVE Protocol, Version 6.0',
 '- Albumin',
 '- Fibrinogen',
 '- CRP',
 '- WBC',
 '- Hemoglobin',
 '- CBC',
 '- Hematocrit',
 '- Platelets',
 '- Creatinine',
 '- BUN',
 '- Na',
 '- K',
 '- Cl',
 '- CO2',
 '- Glucose',
 '- Calcium',
 '- Bilirubin',
 '- AST',
 '- ALT',
 '- DHE

In [23]:
processed = []
for answer in my_answers:
    if not answer == 'No validated clinical questionnaires were used in this text.' and not answer == 'None':
        temp = answer.replace('\n', '')
        temp1 = temp.replace('The validated clinical questionnaires used in this text are', '')
        temp2 = temp1.replace('The validated clinical questionnaires used in this text', '')
        temp3 = temp2.replace('Validated clinical questionnaires used in this text', '')
        temp4 = temp3.strip(':-').strip('- ')
        temp5 = temp4.strip('There are no validated clinical questionnaires are used in this text.')
        processed.append(temp5)
    
processed = list(filter(None, processed))


In [24]:
for answer in processed:
    print(answer)

None f
Expanded Short Physical Performance Battery
med Up and G
Philadelphia Mindfulness S
Montreal Cognitive Assessment (MoCA)
Abbreviated cognitive battery (RBANS)
Falls Efficacy Scale I
Geriatric Depression S
Geriatric Anxiety S
Apathy Evaluation S
Alternative Uses Task
Multisensory Reaction Tim
QOL-AD
Falls Efficacy S
SF-36
Fullerton Advanced Balance S
Zarit Caregiver Burden S
Neuropsychiatric Inventory Q
Postural Sway
Gait Speed and Variability
Abbreviated Cognitive Battery
MRI scan of b
Laboratory P
IMOVE Protocol, Version 6.0
Album
Fibrinog
CRP
WBC
Hemoglob
CBC
Hem
P
C
BUN
N
K
C
CO2
G
Calcium
Bilirub
AS
AL
DHEA-Sulf
I
Cortisol (AM)


EQ-5D, European Quality of Life 5-Dimension Q
DLQI, Dermatology Life Quality I
PGI-S, Patient Global Impression of Severity
PGI-C, Patient Global Impression of Chang
SF-36, Short Form 36
EQ-5D-5L, EuroQol 5 Dimension 5 Level Health Utility I
EQ-VAS, EuroQol Visual Analogue S
WPAI-ANS-CPV, Work Productivity and Activity Impairment Q
CKD-AQ, Symptoms of aCKD
CKD-AQ, Chronic Kidney Disease - Anemia Q
SF-36, Short Form 36 Health Survey
EQ-5D, European Quality of Life 5-Dimension Q
WPAI-ANS-CPV, Work Productivity and Activity Impairment - Anemia Short Form
KDOQI, Kidney Disease Outcomes Quality I
CKD-EPI, CKD Epidemiology Collab
Levey, 2009
Answ
PGI-S, Patient Global Impression of Severity
PGI-C, Patient Global Impression of Chang
SF-36, Health Related Quality of Lif
EQ-5D-5L, EuroQol 5 Dimension 5 Level Health Utility I
EQ-VAS, EuroQol Visual Analogue S
WPAI-ANS-CPV, Work Productivity and Activity Impairment Questionnaire for the specific health problem of anemia, Clinical Practice Version 2.0
SF-36 Vitality scale,
SF-36 Subscales,
EuroQol 5 Dimension 5 Level Health Utility Index,
EuroQol Visual Analogue Scale,
Chronic Kidney Disease - Anemia Q
WPAI-ANS-CPV, Work Productivity and Activity Impairment Questionnaire: Anemic Symptoms Clinical Practice V
SF-36, Short Form -36


chatgpt
SF-36, 36-item Short Form Health Survey Vitality dom
EQ-5D, European Quality of Life 5-Dimension Q
DLQI, Dermatology Life Quality I
EQ-5D-5L, EuroQol 5 Dimension 5 Level Health Utility I
DLQI, Dermatology Life Quality I
PGI-S, Patient Global Impression of Severity
PGI-C, Patient Global Impression of Chang
CKD-AQ8, Symptoms of aCKD
SF-36, Short Form 36
EQ-VAS, EuroQol Visual Analogue S
WPAI-ANS-CPV, Work Productivity and Activity Impairment Q
CKD-AQ, Chronic Kidney Disease - Anemia Q
EQ-5D-5L, European Quality of Life 5-Dimension 5-Level Q
SF-36, Short Form-36 Health Survey
WPAI-ANS-CPV, Work Productivity and Activity Impairment - Allergy Specific and Child Productivity and Activity Impairment Q
KDOQI CKD stages 3, 4 or 5, Kidney Disease Outcomes Quality I
HemoCue Hgb, Stab
Females of childbearing potential (FCBP)
Hemocue Hgb
IV
O
FDA, Food and Drug Adm
There are no validated clinical questionnaires mentioned in the p
There are no validated clinical questionnaires mentioned in the g
CKD-AQ, Chronic Kidney Disease - Anemia Q
SF-36, Health Related Quality of Lif
EQ-5D-5L, Health S
EQ-VAS, EuroQol Visual Analogue S
WPAI-ANS-CPV, Work Productivity and Activity Impairment Questionnaire for the specific health problem of anemia, Clinical Practice Version 2.0
SF-36, Short Form Health Survey
EQ-5D-5L, EuroQol 5 Dimension 5 Level Health Utility I
WPAI-ANS-CPV, Work Productivity and Activity Impairment Questionnaire: Anemic Symptoms Clinical Practice V
SF-36, Short Form -36
PGI-C, Patient Global Impression of Chang
PGI-S, Patient Global Impression of Severity
There are no validated clinical questionnaires m
